In [1]:
import dash
from dash.dependencies import Output, Event, Input
import dash_core_components as dcc
import dash_html_components as html
import plotly
import random
import plotly.graph_objs as go
from collections import deque
import sqlite3
import pandas as pd
import time
import numpy as np

In [ ]:
#google, trump, gun, usa


pos_neg = deque(maxlen=1)
#df_tweets = pd.DataFrame()
#pos = 0
#neg = 0

app = dash.Dash(__name__)
app.layout = html.Div(
    [   html.H2('ProjectX - Live Twitter Sentiment'),
        dcc.Input(id='sentiment_term', value='Trump', type='text'),
        dcc.Graph(id='live-graph', animate=False),
        dcc.Interval(
            id='graph-update',
            interval=1*1000
        ),
     dcc.Graph(id='sentiment-pie', animate=False),
     dcc.Interval(
             id='sentiment-pie-update',
             interval=5*1000
     ),
     
    ],

    
    
)



@app.callback(Output('live-graph', 'figure'),
              [Input(component_id='sentiment_term', component_property='value')],
              events=[Event('graph-update', 'interval')])
def update_graph_scatter(sentiment_term):
    try:
        conn = sqlite3.connect('twitter.db')
        c = conn.cursor()
        df = pd.read_sql("SELECT * FROM sentiment WHERE tweet LIKE ? ORDER BY unix DESC LIMIT 1000", conn ,params=('%' + sentiment_term + '%',))
        df.sort_values('unix', inplace=True)
        #df2 = pd2.read_sql("SELECT * FROM sentiment WHERE tweet LIKE ? ORDER BY unix DESC LIMIT 1000", conn ,params=('%' + sentiment_term + '%',))
        #df2.sort_values('unix', inplace=True)
        
        df['sentiment_smoothed'] = df['sentiment'].rolling(int(len(df)/2)).mean()
        #df2['sentiment_smoothed'] = df2['sentiment'].rolling(int(len(df2)/2)).mean()
        
        pos = df['sentiment'].gt(0).sum()
        neg = df['sentiment'].lt(0).sum()
        
        pos_neg.append([pos,neg])
        
        df['date'] = pd.to_datetime(df['unix'],unit='ms')
        df.set_index('date', inplace=True)
        
        df = df.resample('1s').mean()
        df.dropna(inplace=True)
        X = df.index
        Y = df.sentiment_smoothed
        
        data = plotly.graph_objs.Scatter(
                x=X,
                y=Y,
                name='Scatter',
                mode= 'lines+markers'
                )
        print('updating graph 1')

        return {'data': [data],'layout' : go.Layout(xaxis=dict(range=[min(X),max(X)]),
                                                    yaxis=dict(range=[min(Y),max(Y)]),
                                                    title='Term: {}'.format(sentiment_term))}

    except Exception as e:
        with open('errors.txt','a') as f:
            f.write(str(e))
            f.write('\n')

            
@app.callback(Output('sentiment-pie', 'figure'),
              [Input(component_id='sentiment_term', component_property='value')],
              events=[Event('sentiment-pie-update', 'interval')])
def update_pie_chart(sentiment_term): 
    try:
        #conn = sqlite3.connect('twitter.db')
        #c = conn.cursor()
        #df2 = pd.read_sql("SELECT * FROM sentiment WHERE tweet LIKE ? ORDER BY unix DESC LIMIT 1000", conn ,params=('%' + sentiment_term + '%',))
        #df2['sentiment_smoothed'] = df2['sentiment'].rolling(int(len(df2)/2)).mean()
        
        #pos = df2['sentiment_smoothed'].gt(0).sum()
        #neg = df2['sentiment_smoothed'].lt(0).sum()
        
        pos = pos_neg[0][0]
        neg = pos_neg[0][1]
        
        labels = ['Positive','Negative']

        
        values = [pos/(pos+neg),neg/(pos+neg)]
        
        #print(pos,neg,values)
        colors = ['#007F25', '#800000']

        trace = go.Pie(labels=labels, values=values,
                       hoverinfo='label+percent', textinfo='value', 
                       textfont=dict(size=20),
                       marker=dict(colors=colors))
        print('updating graph 2')

        
        return {"data":[trace],'layout' : go.Layout(
            title='Positive vs Negative sentiment for "{}" '.format(sentiment_term),
            showlegend=True)}
    
    except Exception as e:
        print(e)
        #with open('errors_pie.txt','a') as f:
        #    f.write(str(e))
        #    f.write('\n')

            
            
if __name__ == '__main__':
    app.run_server(debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [12/Dec/2018 12:32:05] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2018 12:32:06] "POST /_dash-update-component HTTP/1.1" 200 -


deque index out of range


127.0.0.1 - - [12/Dec/2018 12:32:06] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2018 12:32:06] "POST /_dash-update-component HTTP/1.1" 200 -


updating graph 1


...

127.0.0.1 - - [12/Dec/2018 12:36:46] "POST /_dash-update-component HTTP/1.1" 200 -


updating graph 1


127.0.0.1 - - [12/Dec/2018 12:36:48] "POST /_dash-update-component HTTP/1.1" 200 -


updating graph 2
updating graph 1


127.0.0.1 - - [12/Dec/2018 12:36:49] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2018 12:36:49] "POST /_dash-update-component HTTP/1.1" 200 -
